This is a simple notebook to explore Gradient Descent methods linear data with some (non-Gaussian) scatter.

It accompanies Chapter 5 of the book (2 of 5).

Author: Viviana Acquaviva, with contributions by Jake Postiglione and Olga Privman.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline

font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
matplotlib.rcParams.update({'figure.autolayout': False})
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
from sklearn import linear_model

In [ ]:
model = linear_model.LinearRegression()

In [ ]:
np.random.seed(16) #set seed for reproducibility purposes

x = np.arange(100) 

yp = 3*x + 3 + 2*(np.random.poisson(3*x+3,100)-(3*x+3)) #generate some data with scatter following Poisson distribution 
                                                       #with exp value = y from linear model, centered around 0

### We can use our data set with outliers from the previous notebook.

In [ ]:
np.random.seed(12) #set 
out = np.random.choice(100,15) #select 15 outliers indexes
yp_wo = np.copy(yp)
np.random.seed(12) #set again
yp_wo[out] = yp_wo[out] + 5*np.random.rand(15)*yp[out]

In [ ]:
plt.scatter(x,yp_wo)
plt.scatter(x,yp)

We can see the effect for the MSE loss right away:

In [ ]:
model.fit(x.reshape(-1,1),yp_wo)

slope, intercept = model.coef_, model.intercept_

print(slope, intercept)

### Learning Check-in
    
What is the main difference in the slope and intercept you found, compared to the case of no outliers?

<details>
<summary style="display: list-item;">Click here for the answer!</summary>
<p>
    
```
The slope changes noticeably, from ~3 to ~4, because the outliers heavily affect the MSE loss. The intercepts also changes significantly (but remember that the intercept is much harder to pinpoint in a linear problem).
```
    
</p>
</details>

### Let's now implement the simplest form of gradient descent: batch, stochastic, and mini-batch, one by one.

In [ ]:
X = np.c_[np.ones((100, 1)), x] # add x0 = 1 to each instance; this is the bias term

print(X.shape) #shape is number of instances x number of parameters

In [ ]:
theta_ne = np.array([[1.548],[3.978]])

In [ ]:
loss_ne = np.mean((X.dot(theta_ne) - yp_wo.reshape(-1,1))**2)

In [ ]:
loss_ne

### Batch GD

In [ ]:
np.random.seed(10) #same initial conditions for all

eta = 0.0001
n_iterations = 1000 #try changing this number!
m = 100

theta_path_bgd = []

theta = np.random.randn(2,1)

for iteration in range(n_iterations):
    gradients = 2/m * X.T.dot(X.dot(theta) - yp_wo.reshape(-1,1))
    theta = theta - eta * gradients
    theta_path_bgd.append(theta)

theta_path_bgd = np.array(theta_path_bgd) #save the path

theta_bgd = theta #final result

In [ ]:
theta_bgd

In [ ]:
loss_bgd = np.sum(1/m*(X.dot(theta_bgd) - yp_wo.reshape(-1,1))**2)

In [ ]:
loss_bgd

### Learning Check-in

What is the percent difference between the final value of the loss found by BGD and by the normal equation?

<details>
<summary style="display: list-item;">Click here for the answer!</summary>
<p>
    
```
(loss_ne-loss_bgd)/loss_ne*100

(it should be of the order of 10^-5, showing that BGD and NE are essentially equivalent.)
```
    
</p>
</details>

What happens to this comparison if you increase the number of iterations in BGD?? 

<details>
<summary style="display: list-item;">Click here for the answer!</summary>
<p>
    
```
They should get closer.
```

### Stochastic GD

In [ ]:
np.random.seed(10) #same initial conditions for all

theta = np.random.randn(2,1) 

eta = 0.000005

n_iterations = 10000 #more iterations

theta_path_sgd = []

for epoch in range(n_iterations):
    
        random_index = np.random.randint(m) # pick one example from the data 
        
        x_one = X[random_index:random_index+1]
        
        y_one = yp_wo[random_index:random_index+1]
        
        gradients = 2 * x_one.T.dot(x_one.dot(theta) - y_one)
        theta = theta - eta * gradients
        theta_path_sgd.append(theta)                 

theta_path_sgd = np.array(theta_path_sgd)

theta_sgd = theta

In [ ]:
theta_sgd

Again, we find a similar theta, but not exactly the same.

In [ ]:
loss_sgd = np.sum(1/m*(X.dot(theta_sgd) - yp_wo.reshape(-1,1))**2)

In [ ]:
loss_sgd

In [ ]:
(loss_ne-loss_sgd)/loss_sgd*100 #percent difference with normal equation

### Learning Check-in
    
Should we be worried that the final value of the loss for the SGD is not that close to the one found by the Normal equation?

<details>
<summary style="display: list-item;">Click here for the answer!</summary>
<p>
    
```
No, because we know that the statistical fluctuations of the SGD algorithms are large, and the loss is not guaranteed to decrease at every step.
```

### Mini batch GD

In [ ]:
# See also implementation notes here: https://sebastianraschka.com/faq/docs/sgd-methods.html

np.random.seed(10)

theta = np.random.randn(2,1) 

eta = 0.000005

n_iterations = 1000

theta_path_mgd = []

minibatch_size = 10 #size of the mini batch

for epoch in range(n_iterations):
    
    shuffled_indices = np.random.permutation(m) #shuffle array 
    
    X_shuffled = X[shuffled_indices]
    
    y_shuffled = yp_wo.reshape(-1,1)[shuffled_indices]
    
    xi = X_shuffled[:minibatch_size]
    
    yi = y_shuffled[:minibatch_size]
    
    gradients = 2/minibatch_size * xi.T.dot(xi.dot(theta) - yi)
    
    theta = theta - eta * gradients
    
    theta_path_mgd.append(theta)

theta_path_mgd = np.array(theta_path_mgd)

theta_mgd = theta 

print(theta_mgd)

In [ ]:
loss_mgd = np.sum(1/m*(X.dot(theta_mgd) - yp_wo.reshape(-1,1))**2)

In [ ]:
loss_mgd

In [ ]:
(loss_ne-loss_mgd)/loss_ne*100 #percent difference with normal equation

Same as before.

It's most interesting to actually look at the path taken by GD in the three cases. Increasingly dark colors denote later steps. 

In [ ]:
plt.figure(figsize=(14,8))

plt.scatter(theta_path_sgd[::10, 0].flatten(), theta_path_sgd[::10, 1].flatten(), marker = 's', s = 5, \
         label="Stochastic GD, N$_{it}$ = 10000", c = np.arange(1000), cmap=plt.cm.Purples)
plt.scatter(theta_path_mgd[:, 0].flatten(), theta_path_mgd[:, 1].flatten(), marker = "+", s = 12, linewidth=1, \
            label="Mini-batch GD, N$_{it}$ = 1000", c = np.arange(1000), cmap=plt.cm.Greens)
plt.scatter(theta_path_bgd[:, 0].flatten(), theta_path_bgd[:, 1].flatten(), marker = "d", s = 12, linewidth=1, \
            label="Batch GD, N$_{it}$ = 1000", c = np.arange(1000,0,-1), cmap=plt.cm.copper)

plt.scatter(theta_sgd[0],theta_sgd[1], marker = "s", s = 100, color = 'Purple', alpha = 0.5)
plt.scatter(theta_mgd[0],theta_mgd[1], marker = "+", s = 200, color = 'DarkGreen', alpha = 1)
plt.scatter(theta_bgd[0],theta_bgd[1], marker = "d", s = 100, color = 'k', alpha = 0.5)
#plt.text(1.5,3.978,'Normal Equation solution X')

legend = plt.legend(loc="upper left", fontsize=16)


for i in range(3):

    legend.legendHandles[i].set_color('k')
    legend.legendHandles[i]._sizes = [30]

plt.xlabel(r"$\theta_0$", fontsize=20)
plt.ylabel(r"$\theta_1$   ", fontsize=20)

plt.axis([1.3, 1.4, 2.5, 6.5])

#plt.savefig('AllThePaths.png', dpi = 300)
plt.show()


### No outliers

In [ ]:
#Batch GD

np.random.seed(10) #same initial conditions for all

eta = 0.0001
n_iterations = 1000
m = 100

theta_path_bgd = []

theta = np.random.randn(2,1)

for iteration in range(n_iterations):
    gradients = 2/m * X.T.dot(X.dot(theta) - yp.reshape(-1,1))
    theta = theta - eta * gradients
    theta_path_bgd.append(theta)

theta_path_bgd = np.array(theta_path_bgd)

#Stochastic GD:

np.random.seed(10) #same initial conditions for all

theta = np.random.randn(2,1) 

eta = 0.00005

n_iterations = 1000

theta_path_sgd = []

for epoch in range(n_iterations):
    
        random_index = np.random.randint(m) # pick one example from the data 
        xi = X[random_index:random_index+1]
        yi = yp[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        theta = theta - eta * gradients
        theta_path_sgd.append(theta)                 # not shown

theta_path_sgd = np.array(theta_path_sgd)


#Mini batch GD:

np.random.seed(10)

theta = np.random.randn(2,1) 

eta = 0.0001

n_iterations = 1000

theta_path_mgd = []

minibatch_size = 10

for epoch in range(n_iterations):
    
    shuffled_indices = np.random.permutation(m) #shuffle array 
    X_shuffled = X[shuffled_indices]
    y_shuffled = yp.reshape(-1,1)[shuffled_indices]
    
    xi = X_shuffled[0:minibatch_size] #without replacement, technically I should
    yi = y_shuffled[0:minibatch_size]
    gradients = 2/minibatch_size * xi.T.dot(xi.dot(theta) - yi)
    theta = theta - eta * gradients
    theta_path_mgd.append(theta)

theta_path_mgd = np.array(theta_path_mgd)

In [ ]:
plt.figure(figsize=(14,8))
plt.scatter(theta_path_bgd[:, 0].flatten(), theta_path_bgd[:, 1].flatten(), marker = "d", s = 12, linewidth=1, \
            label="Batch", c = np.arange(1000,0,-1), cmap=plt.cm.copper)
plt.scatter(theta_path_sgd[:, 0].flatten(), theta_path_sgd[:, 1].flatten(), marker = 's', s = 5, \
         label="Stochastic", c = np.arange(1000), cmap=plt.cm.Purples)
plt.scatter(theta_path_mgd[:, 0].flatten(), theta_path_mgd[:, 1].flatten(), marker = "+", s = 12, linewidth=1, \
            label="Mini-batch", c = np.arange(1000), cmap=plt.cm.Greens)
legend = plt.legend(loc="upper left", fontsize=16)

for i in range(3):

    legend.legendHandles[i].set_color('k')
    legend.legendHandles[i]._sizes = [30]

plt.show()

Exercise 1: note what happens for larger learning rates and smaller learning rates. Would an adaptive learning rate be a solution? Qualitatively, how would you choose it?

Exercise 2: Examine the gradients to discover why batch GD stops updating the slope pretty quickly. Would this be a concern in terms of getting stuck on local minima (in loss function that are not convex)?